In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
def scale_features(df: pd.DataFrame, feature_names: list) -> pd.DataFrame:
    """Scale specified integer features using MinMaxScaler."""
    scaler = MinMaxScaler()
    df[feature_names] = scaler.fit_transform(df[feature_names])
    return df

def classify_features(df: pd.DataFrame) -> pd.DataFrame:
    """Classify features into Binary, Integer, Continuous, or Unknown."""
    summary = []
    for col in df.columns:
        col_data = df[col].dropna()
        unique_vals = set(col_data.unique())
        is_binary = unique_vals.issubset({0, 1})
        is_integer = np.all(np.equal(np.mod(col_data, 1), 0))
        
        if is_binary:
            feature_type = "Binary"
            details = f"Values: {sorted(unique_vals)}"
        elif is_integer:
            feature_type = "Integer"
            details = f"Range: {col_data.min()} to {col_data.max()}"
        elif not is_integer and col_data.dtype.kind in 'fi':  # float or integer
            feature_type = "Continuous"
            details = f"Range: {col_data.min()} to {col_data.max()}"
        else:
            feature_type = "Unknown"
            details = f"Values: {sorted(unique_vals)}"
        
        summary.append({
            'Feature': col,
            'Type': feature_type,
            'Details': details
        })

    return pd.DataFrame(summary)

data = pd.read_csv('../data/all_data.csv')

# Classify feature types
feature_summary = classify_features(data)
print(feature_summary)

# Find integer features to scale
integer_features = feature_summary[feature_summary['Type'] == 'Integer']['Feature'].tolist()
integer_features = [feat for feat in integer_features if feat != 'persoon_leeftijd_bij_onderzoek']

# Scale integer features
data = scale_features(data, integer_features)

                                      Feature        Type  \
0                      adres_aantal_brp_adres     Integer   
1           adres_aantal_verschillende_wijken     Integer   
2                   adres_aantal_verzendadres     Integer   
3            adres_aantal_woonadres_handmatig     Integer   
4                        adres_dagen_op_adres     Integer   
..                                        ...         ...   
313  typering_transport__logistiek___tuinbouw      Binary   
314       typering_zorg__schoonmaak___welzijn      Binary   
315                                        Ja  Continuous   
316                                       Nee  Continuous   
317                                   checked      Binary   

                                           Details  
0                                   Range: 1 to 12  
1                                    Range: 1 to 7  
2                                    Range: 0 to 3  
3                                    Range: 0 to 3  
4 

In [7]:
#data.to_csv(f"../data/all_data_scaled_baseline.csv", index=False)

In [8]:

# Prepare features and target
y = data[['Ja', 'checked']]
X = data.drop(['Ja', 'Nee', 'checked'], axis=1).astype(np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [9]:
# Define and train the logistic regression model
# Define the regularization strength (alpha)
# This can be tuned using cross-validation

ridge_alpha = 1.0  # Example value; you may want to optimize this

# Create a Ridge regressor with the specified alpha
#regressor = Ridge(
#    alpha=ridge_alpha,
#    fit_intercept=True,
#    copy_X=True,
#    random_state=42
#)

regressor = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

pipeline = Pipeline([('regression', regressor)])

pipeline.fit(
    X_train, y_train['Ja']
)

threshold = (0.697021996059818 + 0.697013377682873) / 2.0  #approx. boundary value in dataset

# Predict and evaluate
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test['checked'], y_pred > threshold)
print(f"Model accuracy: {accuracy:.2%}")

Model accuracy: 92.90%


In [10]:
import seaborn as sns

linear_model = pipeline.named_steps['regression']

# Get the coefficients from the model
coefficients = linear_model.coef_

# Get feature names from X_train
feature_names = X_train.columns

# Create a DataFrame mapping feature names to their coefficients
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Calculate absolute coefficients for sorting
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()

# Sort the DataFrame by absolute coefficient values in descending order
coef_df_sorted = coef_df.sort_values(by='Abs_Coefficient', ascending=False)

# Select the top 30 features
top_30_coef = coef_df_sorted.head(30)

print(top_30_coef)

# Optional: Sort the top 30 for better visualization (e.g., ascending order)
#top_30_coef = top_30_coef.sort_values(by='Coefficient')

# print coefficient for 'persoonlijke_eigenschappen_taaleis_voldaan' from coef_df
print(coef_df[coef_df['Feature'] == 'persoonlijke_eigenschappen_taaleis_voldaan'])
print(coef_df[coef_df['Feature'] == 'persoon_leeftijd_bij_onderzoek'])

# Set the plot style for better aesthetics
sns.set_theme(style='whitegrid')

# Initialize the matplotlib figure
plt.figure(figsize=(10, 6))

# Create a horizontal bar plot
sns.barplot(
    x='Coefficient',
    y='Feature',
    data=top_30_coef,
    palette='viridis'
)

# Add title and labels
plt.title('Top 30 Feature Coefficients in Ridge Regression Model', fontsize=16)
plt.xlabel('Coefficient Value', fontsize=14)
plt.ylabel('Feature', fontsize=14)

# Improve layout
plt.tight_layout()

# Display the plot
plt.show()

AttributeError: 'GradientBoostingRegressor' object has no attribute 'coef_'

### Store to ONNX

In [12]:
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from skl2onnx import convert_sklearn
from onnx import helper
import onnxruntime as rt
import onnx

In [13]:
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

In [14]:
# Add note to onnx graph such that it saves both the predicted risk value
# and the 'checked' value based on if the risk value is higher than the
# provided threshold.

graph = onnx_model.graph
threshold_node = helper.make_node(
    "Constant",
    inputs=[],
    outputs=["threshold"],
    value=helper.make_tensor("value", onnx.TensorProto.FLOAT, [], [threshold])
)
graph.node.append(threshold_node)

greater_node = helper.make_node(
    "Greater",
    inputs=[graph.output[0].name, "threshold"],
    outputs=["boolean_output"]
)
graph.node.append(greater_node)

boolean_output = helper.make_tensor_value_info("boolean_output", onnx.TensorProto.BOOL, [None])
graph.output.extend([boolean_output])

In [15]:
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

# y_pred_onnx[0] = risk values
# y_pred_onnx[1] = boolean value indicating if high risk or not
accuracy_onnx_model = accuracy_score(y_test['checked'], y_pred_onnx[1])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.9289846153846154


In [16]:
onnx.save(onnx_model, "../model/gboost_regression_model_baseline_all_data.onnx")